In [ ]:
!apt-get update
!apt-get install hisat2
!hisat2 --version

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
!apt-get install sra-toolkit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sra-toolkit is already the newest version (2.8.2-5+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [ ]:
!java -version

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [ ]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip

--2021-12-01 23:46:00--  https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
Resolving www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)... 149.155.133.4
Connecting to www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)|149.155.133.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10249221 (9.8M) [application/zip]
Saving to: ‘fastqc_v0.11.9.zip.1’

fastqc_v0.11.9.zip. 100%[===================>]   9.77M  3.03MB/s    in 3.2s    

2021-12-01 23:46:04 (3.03 MB/s) - ‘fastqc_v0.11.9.zip.1’ saved [10249221/10249221]



In [ ]:
!unzip fastqc_v0.11.9.zip

Archive:  fastqc_v0.11.9.zip
replace FastQC/cisd-jhdf5.jar? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Новый раздел

In [ ]:
!chmod a+x FastQC/fastqc

In [ ]:
!pip install HTSeq

     |████████████████████████████████| 1.5 MB 25.3 MB/s 
     |████████████████████████████████| 14.9 MB 37.5 MB/s 


In [ ]:
!pip install --upgrade numpy

In [ ]:
!htseq-count --version

1.99.2


In [ ]:
!wget https://genome-idx.s3.amazonaws.com/hisat/mm10_genome.tar.gz
!tar -xzvf mm10_genome.tar.gz
!wget http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M25/gencode.vM25.annotation.gtf.gz
!gzip -d gencode.vM25.annotation.gtf.gz

--2021-12-02 00:33:11--  https://genome-idx.s3.amazonaws.com/hisat/mm10_genome.tar.gz
Resolving genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)... 52.217.206.1
Connecting to genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)|52.217.206.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3804366597 (3.5G) [binary/octet-stream]
Saving to: ‘mm10_genome.tar.gz’

mm10_genome.tar.gz  100%[===================>]   3.54G  28.2MB/s    in 1m 55s  

2021-12-02 00:35:06 (31.5 MB/s) - ‘mm10_genome.tar.gz’ saved [3804366597/3804366597]

mm10/
mm10/genome.8.ht2
mm10/genome.5.ht2
mm10/make_mm10.sh
mm10/genome.7.ht2
mm10/genome.6.ht2
mm10/genome.4.ht2
mm10/genome.3.ht2
mm10/genome.1.ht2
mm10/genome.2.ht2
--2021-12-02 00:35:59--  http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M25/gencode.vM25.annotation.gtf.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:80... connected.
HTT

In [ ]:
data = ['SRR3414629', 'SRR3414630', 'SRR3414631', 'SRR3414635', 'SRR3414636', 'SRR3414637']

In [ ]:
for cur in data:
    %env cur=$cur
    !time  fastq-dump --split-files $cur
    !echo /content/"$cur"_1.fastq
    !./FastQC/fastqc /content/"$cur"_1.fastq
    !time hisat2 -x -p 3 mm10/genome -U "$cur"_1.fastq -S "$cur"_1.sam 2> "$cur".hisat
    !grep -P '^@|NH:i:1$' "$cur"_1.sam > "$cur".uniq.sam
    !grep -v '^@' "$cur".uniq.sam | wc -l > "cur".result
    !time htseq-count --stranded=no --format=sam "$cur".uniq.sam gencode.vM25.annotation.gtf > "$cur".counts
    !grep '^__' "$cur".counts >> "$cur".result
    !rm -v "$cur".uniq.sam "$cur"_1.fastq "$cur"_1_fastqc.zip "$cur"_1.sam

env: cur=SRR3414629
Read 21106089 spots for SRR3414629
Written 21106089 spots for SRR3414629

real	3m38.648s
user	2m56.479s
sys	0m8.332s
/content/SRR3414629_1.fastq
Started analysis of SRR3414629_1.fastq
Approx 5% complete for SRR3414629_1.fastq
Approx 10% complete for SRR3414629_1.fastq
Approx 15% complete for SRR3414629_1.fastq
Approx 20% complete for SRR3414629_1.fastq
Approx 25% complete for SRR3414629_1.fastq
Approx 30% complete for SRR3414629_1.fastq
Approx 35% complete for SRR3414629_1.fastq
Approx 40% complete for SRR3414629_1.fastq
Approx 45% complete for SRR3414629_1.fastq
Approx 50% complete for SRR3414629_1.fastq
Approx 55% complete for SRR3414629_1.fastq
Approx 60% complete for SRR3414629_1.fastq
Approx 65% complete for SRR3414629_1.fastq
Approx 70% complete for SRR3414629_1.fastq
Approx 75% complete for SRR3414629_1.fastq
Approx 80% complete for SRR3414629_1.fastq
Approx 85% complete for SRR3414629_1.fastq
Approx 90% complete for SRR3414629_1.fastq
Approx 95% complete for

In [ ]:
import pandas as pd

prog, control = data[:3], data[3:]

cols = []
for d, c in zip([control, prog], ['c', 'r']):
  for i, file in enumerate(d):
    cur_csv = pd.read_csv(file + '.counts', sep='\t', header=None, index_col=0)
    cols.append(cur_csv)
    cols[-1].columns = [c + str(i + 1)]

drop_features = [
  '__no_feature',
  '__ambiguous',
  '__too_low_aQual',
  '__not_aligned',
  '__alignment_not_unique'
]

overall = pd.concat(cols, axis=1)
for feature in drop_features:
  overall.drop(feature, inplace=True)

overall.to_csv('ALL.counts', sep='\t', index_label=True)